# Employee Feedback Sentiment Analysis

In [7]:
!pip install transformers torch numpy pandas scikit-learn textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.6 MB/s eta 0:00:00


### Response sentiment analysis

In [31]:
from transformers import pipeline

# Load the sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

# Example feedback data
feedback_data = [
    "I really enjoy the team activities and the work environment.",
    "I feel like my work has a deep impact, but it's stressful.",
    "I'm content with the management, but I don't find my tasks very engaging.",
    "I really appreciate the new flexible work policies.",
    "The management needs to be more transparent about decisions.",
    "This is the worst work environment I've experienced.",
    "Great, thanks."
]

# Analyze sentiment
sentiment_results = [sentiment_analyzer(feedback) for feedback in feedback_data]

# Display sentiment results
for i, result in enumerate(sentiment_results):
    print(f"Feedback: {feedback_data[i]}")
    print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']}\n")

# Leave only numeric sentiment results
sentiment_results_clean = [result[0]['score'] for result in sentiment_results]
print(sentiment_results_clean)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Feedback: I really enjoy the team activities and the work environment.
Sentiment: POSITIVE, Score: 0.9998564720153809

Feedback: I feel like my work has a deep impact, but it's stressful.
Sentiment: POSITIVE, Score: 0.9902501702308655

Feedback: I'm content with the management, but I don't find my tasks very engaging.
Sentiment: NEGATIVE, Score: 0.997974693775177

Feedback: I really appreciate the new flexible work policies.
Sentiment: POSITIVE, Score: 0.9998334646224976

Feedback: The management needs to be more transparent about decisions.
Sentiment: NEGATIVE, Score: 0.9981219172477722

Feedback: This is the worst work environment I've experienced.
Sentiment: NEGATIVE, Score: 0.9998093247413635

Feedback: Great, thanks.
Sentiment: POSITIVE, Score: 0.9998507499694824

[0.9998564720153809, 0.9902501702308655, 0.997974693775177, 0.9998334646224976, 0.9981219172477722, 0.9998093247413635, 0.9998507499694824]


### Embeddings Creation

In [32]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.cluster import KMeans
import numpy as np

# Load tokenizer and model for embeddings
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

# Function to get sentence embeddings
def get_embeddings(text_list):
    embeddings = []
    for text in text_list:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        outputs = model(**inputs)
        # Take the mean of the last hidden state
        embedding = torch.mean(outputs.last_hidden_state, dim=1).detach().numpy()
        embeddings.append(embedding.flatten())
    return np.array(embeddings)

# Get embeddings for feedback data
feedback_embeddings = get_embeddings(feedback_data)

# Perform topic modeling with KMeans
num_topics = 3  # Define the number of topics you want to extract
kmeans = KMeans(n_clusters=num_topics, random_state=42)
kmeans.fit(feedback_embeddings)

# Print topic assignments
for i, label in enumerate(kmeans.labels_):
    print(f"Feedback: {feedback_data[i]} -> Topic {label}")

Feedback: I really enjoy the team activities and the work environment. -> Topic 2
Feedback: I feel like my work has a deep impact, but it's stressful. -> Topic 0
Feedback: I'm content with the management, but I don't find my tasks very engaging. -> Topic 0
Feedback: I really appreciate the new flexible work policies. -> Topic 2
Feedback: The management needs to be more transparent about decisions. -> Topic 0
Feedback: This is the worst work environment I've experienced. -> Topic 0
Feedback: Great, thanks. -> Topic 1


### Conductivity score

In [33]:
from sklearn.metrics.pairwise import cosine_similarity

# Predefined meaningful feedback for comparison (templates)
meaningful_feedback_templates = [
    "The policies in place are effective and beneficial for the team.",
    "Management should improve transparency for better team morale.",
    "The work environment is supportive and collaborative."
]
template_embeddings = get_embeddings(meaningful_feedback_templates)

# Function to calculate meaningfulness score
def conductivity_score(feedback_embeddings, template_embeddings):
    scores = []
    for feedback in feedback_embeddings:
        # Calculate similarity to the templates and average the score
        similarities = cosine_similarity([feedback], template_embeddings).flatten()
        meaningfulness = similarities.mean()
        scores.append(meaningfulness)
    return scores

# Calculate meaningfulness scores for feedback
meaningfulness_scores = conductivity_score(feedback_embeddings, template_embeddings)

for (score, feedback) in zip(meaningfulness_scores, feedback_data):
    print(f"Feedback: {feedback}")
    print(f"Meaninfulness Score: {score:.2f}\n")

Feedback: I really enjoy the team activities and the work environment.
Meaninfulness Score: 0.79

Feedback: I feel like my work has a deep impact, but it's stressful.
Meaninfulness Score: 0.73

Feedback: I'm content with the management, but I don't find my tasks very engaging.
Meaninfulness Score: 0.72

Feedback: I really appreciate the new flexible work policies.
Meaninfulness Score: 0.82

Feedback: The management needs to be more transparent about decisions.
Meaninfulness Score: 0.85

Feedback: This is the worst work environment I've experienced.
Meaninfulness Score: 0.70

Feedback: Great, thanks.
Meaninfulness Score: 0.64



### Complexity Analysis

In [34]:
import textstat

# Function to add text complexity check
def complexity_analysis(feedback):
    return textstat.flesch_reading_ease(feedback)

# Calculate text complexity scores
complexity_scores = [complexity_analysis(feedback) for feedback in feedback_data]

# Example of integrating text complexity into analysis
for (score, feedback) in zip(complexity_scores, feedback_data):
    print(f"Feedback: {feedback}")
    print(f"Complexity Score: {score:.2f}\n")

Feedback: I really enjoy the team activities and the work environment.
Complexity Score: 35.95

Feedback: I feel like my work has a deep impact, but it's stressful.
Complexity Score: 93.14

Feedback: I'm content with the management, but I don't find my tasks very engaging.
Complexity Score: 75.20

Feedback: I really appreciate the new flexible work policies.
Complexity Score: 29.52

Feedback: The management needs to be more transparent about decisions.
Complexity Score: 53.88

Feedback: This is the worst work environment I've experienced.
Complexity Score: 46.44

Feedback: Great, thanks.
Complexity Score: 120.21



### Weighted Scoring

In [35]:
def weighted_score(sentiment_results, conductivity_scores, complexity_scores, alpha=0.5, beta=0.3, gamma=0.2):
    """
    Calculates a weighted score using sentiment, conductivity, and complexity scores.

    Parameters:
    - sentiment_results: List of sentiment analysis results (dicts).
    - conductivity_scores: List of pre-calculated conductivity scores (floats).
    - complexity_scores: List of pre-calculated complexity scores (floats).
    - alpha, beta, gamma: Weights for sentiment, conductivity, and complexity, respectively.

    Returns:
    - List of combined weighted scores (floats).
    """
    # Normalize conductivity and complexity scores to a range of 0 to 1
    norm_conductivity_scores = (conductivity_scores - np.min(conductivity_scores)) / (np.max(conductivity_scores) - np.min(conductivity_scores) + 1e-5)
    norm_complexity_scores = (complexity_scores - np.min(complexity_scores)) / (np.max(complexity_scores) - np.min(complexity_scores) + 1e-5)

    # Invert complexity scores so that higher complexity yields a higher contribution
    inverted_complexity_scores = 1 - norm_complexity_scores

    combined_scores = []
    for i, result in enumerate(sentiment_results):
        sentiment_label = result[0]['label']
        sentiment_score = result[0]['score']

        # Adjust sentiment score based on label
        if sentiment_label == 'POSITIVE':
            sentiment_weighted = sentiment_score  # Positive score as is
        elif sentiment_label == 'NEGATIVE':
            sentiment_weighted = 1 - sentiment_score  # Negative score inverted
        else:
            sentiment_weighted = sentiment_score * 0.5  # Neutral (less impactful)

        # Ensure the sentiment score is scaled between 0 and 1
        sentiment_weighted = max(0, min(1, sentiment_weighted))

        # Calculate final weighted score using the provided formula
        final_score = (alpha * sentiment_weighted +
                       beta * norm_conductivity_scores[i] +
                       gamma * inverted_complexity_scores[i])

        # Normalize the final score to ensure it stays between 0 and 1
        final_score = max(0, min(1, final_score))

        combined_scores.append(final_score)

    return combined_scores

# Call the function
weighted_scores = weighted_score(sentiment_results, meaningfulness_scores, complexity_scores, alpha = 0.2, beta=0.5, gamma=0.3)
for i, score in enumerate(weighted_scores):
    print(f"Feedback {i+1}: {feedback_data[i]}")
    print(f"Sensitivity Score (raw) = {sentiment_results[i][0]['label']}: {sentiment_results[i][0]['score']:.4f}")
    print(f"Conductivity Score (raw) = {meaningfulness_scores[i]:.2f}")
    print(f"Complexity Score (raw) = {complexity_scores[i]:.2f}")
    print(f"-----------------= RESULT =-----------------")
    print(f"Weighted Score (normalized) = {score:.2f}\n")

Feedback 1: I really enjoy the team activities and the work environment.
Sensitivity Score (raw) = POSITIVE: 0.9999
Conductivity Score (raw) = 0.79
Complexity Score (raw) = 35.95
-----------------= RESULT =-----------------
Weighted Score (normalized) = 0.83

Feedback 2: I feel like my work has a deep impact, but it's stressful.
Sensitivity Score (raw) = POSITIVE: 0.9903
Conductivity Score (raw) = 0.73
Complexity Score (raw) = 93.14
-----------------= RESULT =-----------------
Weighted Score (normalized) = 0.50

Feedback 3: I'm content with the management, but I don't find my tasks very engaging.
Sensitivity Score (raw) = NEGATIVE: 0.9980
Conductivity Score (raw) = 0.72
Complexity Score (raw) = 75.20
-----------------= RESULT =-----------------
Weighted Score (normalized) = 0.34

Feedback 4: I really appreciate the new flexible work policies.
Sensitivity Score (raw) = POSITIVE: 0.9998
Conductivity Score (raw) = 0.82
Complexity Score (raw) = 29.52
-----------------= RESULT =------------

### Outlier Detection

In [36]:
def detect_outliers(weighted_scores, method="iqr", threshold=1.5):
    """
    Detects outliers based on the weighted scores using the specified method.

    Parameters:
    - weighted_scores: List of combined weighted scores (floats).
    - method: The method to use for outlier detection ('iqr' or 'z-score').
    - threshold: The threshold for outlier detection (default is 1.5 for IQR).

    Returns:
    - List of booleans indicating whether each score is an outlier (True) or not (False).
    """
    adjusted_scores = weighted_scores.copy()

    if method == "iqr":
        # Calculate the first and third quartile (Q1, Q3)
        Q1 = np.percentile(weighted_scores, 25)
        Q3 = np.percentile(weighted_scores, 75)
        IQR = Q3 - Q1

        # Define the lower and upper bounds for outliers
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR

        # Adjust outliers
        for i, score in enumerate(weighted_scores):
            if score < lower_bound:
                # Set to 0.1 if closer to the lower bound
                adjusted_scores[i] = 0.1
            elif score > upper_bound:
                # Set to 0.9 if closer to the upper bound
                adjusted_scores[i] = 0.9

    elif method == "z-score":
        # Calculate the mean and standard deviation
        mean = np.mean(weighted_scores)
        std_dev = np.std(weighted_scores)

        # Adjust outliers based on the z-score
        for i, score in enumerate(weighted_scores):
            z_score = abs((score - mean) / (std_dev + 1e-5))
            if z_score > threshold:
                # Check if the score is closer to 0.1 or 0.9 and adjust accordingly
                if score < mean:
                    adjusted_scores[i] = 0.1
                else:
                    adjusted_scores[i] = 0.9

    else:
        raise ValueError("Invalid method specified. Use 'iqr' or 'z-score'.")

    return adjusted_scores

# Display results
outliers = detect_outliers(weighted_scores)

for i, score in enumerate(outliers):
    print(f"Original Score: {weighted_scores[i]} - Adjusted Score: {score}")

Original Score: 0.8264019590999113 - Adjusted Score: 0.8264019590999113
Original Score: 0.4976503983744206 - Adjusted Score: 0.4976503983744206
Original Score: 0.3446130580001433 - Adjusted Score: 0.3446130580001433
Original Score: 0.9180860340595245 - Adjusted Score: 0.9180860340595245
Original Score: 0.7197705043404908 - Adjusted Score: 0.7197705043404908
Original Score: 0.38605472415147113 - Adjusted Score: 0.38605472415147113
Original Score: 0.199970183073615 - Adjusted Score: 0.199970183073615


### Separation into themes

In [37]:
# Load a pre-trained zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define the categories for classification
categories = [
    "Happiness: How enjoyable people find their day-to-day life at work",
    "Purpose: How meaningful people find their work",
    "Satisfaction: How content people feel with the way things are at work",
    "Stress-free: How manageable people find their work stress"
]

def categorize_feedback(feedback_texts):
    """
    Classifies feedback texts into predefined categories.

    Parameters:
    - feedback_texts: List of feedback texts to be classified.

    Returns:
    - List of dictionaries with category scores for each feedback.
    """
    categorized_results = []
    for feedback in feedback_texts:
        result = classifier(feedback, categories, multi_label=True)
        # Prepare a dictionary mapping each category to its score
        feedback_categories = dict(zip(result['labels'], result['scores']))
        categorized_results.append(feedback_categories)

    return categorized_results

# Example usage
feedback_texts = [
    "I really enjoy the team activities and the work environment.",
    "I feel like my work has a deep impact, but it's stressful.",
    "I'm content with the management, but I don't find my tasks very engaging.",
    "I really appreciate the new flexible work policies.",
    "The management needs to be more transparent about decisions.",
    "This is the worst work environment I've experienced.",
    "Great, thanks."
]

categorized_feedbacks = categorize_feedback(feedback_texts)

# Display categorized feedback
for i, feedback in enumerate(feedback_texts):
    print(f"Feedback: \"{feedback}\"")
    print("Category scores:")
    for category, score in categorized_feedbacks[i].items():
        print(f"  {category}: {score:.2f}")
    print()

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Feedback: "I really enjoy the team activities and the work environment."
Category scores:
  Satisfaction: How content people feel with the way things are at work: 0.99
  Happiness: How enjoyable people find their day-to-day life at work: 0.96
  Stress-free: How manageable people find their work stress: 0.47
  Purpose: How meaningful people find their work: 0.42

Feedback: "I feel like my work has a deep impact, but it's stressful."
Category scores:
  Purpose: How meaningful people find their work: 0.24
  Satisfaction: How content people feel with the way things are at work: 0.02
  Happiness: How enjoyable people find their day-to-day life at work: 0.01
  Stress-free: How manageable people find their work stress: 0.00

Feedback: "I'm content with the management, but I don't find my tasks very engaging."
Category scores:
  Satisfaction: How content people feel with the way things are at work: 0.98
  Happiness: How enjoyable people find their day-to-day life at work: 0.66
  Stress-free: H

In [39]:
import json

def select_best_categories(categorized_feedbacks, feedback_texts, threshold=0.8):
    """
    Selects the best categories from categorized feedback based on the given threshold
    and formats the output as a JSON object with cleaned category labels.

    Parameters:
    - categorized_feedbacks: List of dictionaries with category scores for each feedback.
    - feedback_texts: List of feedback text corresponding to the scores.
    - threshold: The score threshold for including categories (default is 0.8).

    Returns:
    - JSON formatted string with feedback and cleaned selected categories.
    """
    selected_feedback_data = []

    for i, feedback_categories in enumerate(categorized_feedbacks):
        # Find categories that meet or exceed the threshold
        selected_categories = [category.split(':')[0].strip() for category, score in feedback_categories.items() if score >= threshold]

        if not selected_categories:
            # If no category meets the threshold, choose the one with the highest score and clean it
            highest_category = max(feedback_categories, key=feedback_categories.get)
            selected_categories = [highest_category.split(':')[0].strip()]

        # Append to the result list as a dictionary
        selected_feedback_data.append({
            "feedback": feedback_texts[i],
            "categories": selected_categories
        })

    # Convert to JSON format
    return json.dumps(selected_feedback_data, indent=2)

# Get the JSON formatted result
json_result = select_best_categories(categorized_feedbacks, feedback_data)
print(json_result)

[
  {
    "feedback": "I really enjoy the team activities and the work environment.",
    "categories": [
      "Satisfaction",
      "Happiness"
    ]
  },
  {
    "feedback": "I feel like my work has a deep impact, but it's stressful.",
    "categories": [
      "Purpose"
    ]
  },
  {
    "feedback": "I'm content with the management, but I don't find my tasks very engaging.",
    "categories": [
      "Satisfaction"
    ]
  },
  {
    "feedback": "I really appreciate the new flexible work policies.",
    "categories": [
      "Satisfaction"
    ]
  },
  {
    "feedback": "The management needs to be more transparent about decisions.",
    "categories": [
      "Satisfaction"
    ]
  },
  {
    "feedback": "This is the worst work environment I've experienced.",
    "categories": [
      "Satisfaction"
    ]
  },
  {
    "feedback": "Great, thanks.",
    "categories": [
      "Satisfaction"
    ]
  }
]
